In [1]:
import pickle

test_x = []
def get_feat(f):
    with open(f,'rb') as fin:
        a,b = pickle.load(fin)
        return b

feat_files = [
    '../features/glove_cnn_gru_4_feat.pkl',
    '../features/glove_cudnn_gru_4_feat.pkl',
    '../features/glove_lstm_v1_4_feat.pkl',
    '../features/glove_gru_v1_4_feat.pkl'
]
for f in feat_files:
    b = get_feat(f)
    test_x.append(b)

import numpy as np
test_x = np.array(test_x)
print(test_x.shape)
mean_test_x = np.mean(test_x,axis=0)
print(mean_test_x.shape)

(4, 153164, 6)
(153164, 6)


In [2]:
import pandas as pd
sample_submission = pd.read_csv("../input/sample_submission.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission[list_classes] = mean_test_x
sample_submission.to_csv("../results/avg_bagging.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.995861      0.465433  0.965190  0.157851  0.916416   
1  0000247867823ef7  0.000473      0.000004  0.000121  0.000003  0.000058   
2  00013b17ad220c46  0.001627      0.000072  0.000693  0.000064  0.000392   
3  00017563c3f7919a  0.000332      0.000002  0.000076  0.000006  0.000047   
4  00017695ad8997eb  0.007078      0.000040  0.001054  0.000080  0.000575   

   identity_hate  
0       0.497567  
1       0.000016  
2       0.000145  
3       0.000009  
4       0.000124  
save done


In [4]:
def avg_res(res_files):
    test_x = []
    for f in res_files:
        test_x.append(pd.read_csv(f)[list_classes].values)
    return np.mean(test_x,axis=0)

res_fs = ['../results/pub9848.csv','../results/xgb9853.gz']
avg_sub = avg_res(res_fs)
sample_submission[list_classes] = avg_sub
sample_submission.to_csv("../results/avg_sub.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.979936      0.284694  0.952809  0.118262  0.888514   
1  0000247867823ef7  0.000553      0.000122  0.000300  0.000027  0.000249   
2  00013b17ad220c46  0.005728      0.000612  0.002268  0.000131  0.001661   
3  00017563c3f7919a  0.000751      0.000077  0.000289  0.000072  0.000347   
4  00017695ad8997eb  0.004916      0.000114  0.000918  0.000065  0.000824   

   identity_hate  
0       0.443627  
1       0.000117  
2       0.000441  
3       0.000061  
4       0.000108  
save done


In [9]:
def pow_avg_res(res_files):
    test_x = []
    for f in res_files:
        test_x.append(pd.read_csv(f)[list_classes].values)
    file_cnt = len(res_files)
    base = test_x[0]
    for j in range(file_cnt-1):
        for i in range(6):
            base[:,i] = base[:,i]*test_x[j+1][:,i]
    base = base ** (1/file_cnt)
    return base

res_fs = ['../results/pub9848.csv','../results/xgb9853.gz','../results/avg9857.csv.gz']
avg_sub = pow_avg_res(res_fs)
sample_submission[list_classes] = avg_sub
sample_submission.to_csv("../results/pow_avg_sub.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.979815      0.284618  0.952727  0.113660  0.888494   
1  0000247867823ef7  0.000433      0.000087  0.000220  0.000026  0.000142   
2  00013b17ad220c46  0.003804      0.000293  0.001666  0.000103  0.001269   
3  00017563c3f7919a  0.000653      0.000065  0.000245  0.000063  0.000234   
4  00017695ad8997eb  0.004903      0.000095  0.000914  0.000062  0.000776   

   identity_hate  
0       0.442075  
1       0.000095  
2       0.000390  
3       0.000059  
4       0.000107  
save done


In [11]:
# play with weight
df1 = pd.read_csv('../results/pub9848.csv')[list_classes].values
df2 = pd.read_csv('../results/xgb9853.gz')[list_classes].values
play_res = df1*0.3+df2*0.7
sample_submission[list_classes] = play_res
sample_submission.to_csv("../results/weighted_add_sub.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.987482      0.281484  0.958949  0.134110  0.885619   
1  0000247867823ef7  0.000393      0.000083  0.000207  0.000025  0.000159   
2  00013b17ad220c46  0.003801      0.000381  0.001563  0.000093  0.001167   
3  00017563c3f7919a  0.000576      0.000058  0.000217  0.000056  0.000231   
4  00017695ad8997eb  0.005089      0.000084  0.000877  0.000055  0.000690   

   identity_hate  
0       0.461777  
1       0.000085  
2       0.000343  
3       0.000053  
4       0.000099  
save done
